## Neural net zelf bouwen - practicum week 2 / periode 2

In dit notebook documenteer ik de stappen die ik heb gemaakt mbt het zelf implementeren van een neural net. Ik heb voor een notebook gekozen omdat ik hierin de code beter kan onderverdelen en zo mijn stappen/progressie kan bijhouden. 

De opdracht is het implementeren van een neuraal netwerk dat kruisjes van rondjes kan onderscheiden in een 3 x 3 pixels rooster/scherm. Elke pixel kan dus zwart zijn (waarde is dan 1) of wit (waarde is dan 0).

In [6]:
import random
import numpy as np

In [7]:


#Een cirkel ziet er dus zo uit:

cirkel = [
    [1,1,1],
    [1,0,1],
    [1,1,1]
]

#Een kruis ziet er zo uit:

kruis = [
    [0,1,0],
    [1,1,1],
    [0,1,0]
]


Voor nu werk ik enkel met deze twee vormen. Natuurlijk kunnen de negatieven van de bovenstaande matrices ook een kruis/cirkel vormen (maar dan wit van kleur). Daarnaast kan er ook nog een schuin kruis worden gemaakt, maar ook dat laat ik voor nu even links liggen.

## Input

Als input krijgen we dus matrices van 3 x 3. Elk vakje in deze matrix is dus een input in onze inputlaag van totaal 9 neuronen. 

Om te beginnen ga ik voor één enkele neuron de implementatiestappen doorlopen, om dit later uit te breiden naar een heel netwerk.

In [8]:
#De input komt binnen als 3x3 matrix - voor de eenvoud maken we hier een 9x1 vector van:

def transpose_vec(input_vec):
    flat_array = []
    for i in input_vec:
        for n in i:
            flat_array.append([n])
    return flat_array

#Voorbeeld voor hoe dit met een cirkel matrix gaat:
cirkel_vector = transpose_vec(cirkel)

print(cirkel_vector)

[[1], [1], [1], [1], [0], [1], [1], [1], [1]]


Hierboven staat dus input vector die we aan onze 1-cellige NN voeden. 

Vanuit elke input neuron loopt er (in een fully connected netwerk) een weight(lijn) naar elke neuron in de volgende laag (in dit simpele geval direct naar de output laag).

Om tot een output te komen vermenigvuldigen we de input van de input neuron met diens weight. Als we dit voor alle inputs en weights doen, per neuron in de volgende laag, is de output de gewogen som van alle vermenigvuldigingen. Hierbij tellen we de zogenaamde bias bij op. Het resultaat stoppen we vervolgens in een activatie functie, en dit geeft ons de output voor de neuron.

In [9]:
#We initialiseren de weights voor onze neuron. Dit zijn er in ons geval 9.

def init_weights(size):
    return [[round(random.uniform(0, 1), 2)] for i in range(size)]

#Elke input neuron (van de 9) heeft een weight, dus het aantal weights is gelijk aan het aantal input neurons:

weights_voorbeeld = init_weights(len(cirkel_vector))
print(weights_voorbeeld)

[[0.34], [0.88], [0.09], [0.34], [0.08], [0.83], [0.75], [0.39], [0.13]]


Ook de bias zullen we random initialiseren, dit is maar één getal aangezien elke neuron één bias heeft. In een netwerk is de bias een vector met daarin voor elke neuron uit de laag een bias waarde. 

In [10]:
def init_bias():

    return round(random.uniform(0, 1), 2)

bias_voorbeeld = init_bias()
print(bias_voorbeeld)

0.2


We hebben dus nu voor één neuron de volgende waardes:

In [11]:
print(f"Input vector is {cirkel_vector}")
print(f"Weight vector is {weights_voorbeeld}")
print(f"Bias is {bias_voorbeeld}")


Input vector is [[1], [1], [1], [1], [0], [1], [1], [1], [1]]
Weight vector is [[0.34], [0.88], [0.09], [0.34], [0.08], [0.83], [0.75], [0.39], [0.13]]
Bias is 0.2


Nu is het een kwestie van elke waarde in de input vector vermenigvuldigen met de corresponderende weight. Hiervan nemen we de som. De bias tellen we hierbij op, en dat is onze voorlopige output:

In [12]:
#Functie om een vector vlak te maken (dus een enkele list ipv een list of lists)
def flatten_matrix(input_vec):
    
    return [i for subl in input_vec for i in subl]



#Functie om de output te krijgen
def get_output(input_vec, weight_vec, bias):
    
    zipped_l = list(zip(flatten_matrix(input_vec), flatten_matrix(weight_vec)))
    
    return sum([i[0]*i[1] for i in zipped_l]) + bias

We kunnen, als we nog even niet op de activatie functie inzoomen, dus de output van onze neuron berekenen:

In [13]:
print(get_output(cirkel_vector, weights_voorbeeld, bias_voorbeeld))

3.95


## Meerdere neuronen / een laag implementeren

Hierboven hebben we gewerkt met een enkele neuron. Het is nu zaak om te kijken of we een 'layer' van meerdere neuronen kunnen maken en de output hiervan kunnen genereren.

Dit is niets meer dan de bovenstaande stappen volgen voor meerdere setjes weights (per neuron) en biases (per neuron). In dit geval gaan we naar een layer van 2 output neuronen (dus nog geen hidden layer). 

We beginnen met de weights:

In [14]:
#We maken voor elke output neuron een set weights. In dit geval dus twee, maar stel dat we 4 of 5 neuronen willen
#zouden we ook meerdere setjes weights/bias kunnen maken

weights_1 = init_weights(len(cirkel_vector))
weights_2 = init_weights(len(cirkel_vector))

weight_list = [flatten_matrix(weights_1), flatten_matrix(weights_2)]

#Voor elke neuron maken we ook een bias

bias_1 = init_bias()
bias_2 = init_bias()

bias_list = [bias_1, bias_2]

print(weight_list)
print(bias_list)

[[0.38, 0.81, 0.96, 0.98, 0.56, 0.44, 0.82, 0.66, 0.94], [0.95, 0.17, 0.12, 0.33, 0.36, 0.89, 0.27, 0.32, 0.55]]
[0.68, 0.63]


In [15]:
# We zippen de weights en biases waardoor we een lijst hebben van tuples, waarbij elke tuple onze weights en bias bevat)

w_and_b = list(zip(weight_list, bias_list))

In [16]:
for w, b in w_and_b:
    print(f'Weights: {w}, bias: {b}')

Weights: [0.38, 0.81, 0.96, 0.98, 0.56, 0.44, 0.82, 0.66, 0.94], bias: 0.68
Weights: [0.95, 0.17, 0.12, 0.33, 0.36, 0.89, 0.27, 0.32, 0.55], bias: 0.63


De lijst hierboven geeft dus een lijst van tuples, met in elke tuple (elke neuron) de weights en bias.

Hieronder staat een functie die dus één forward pass afwerkt, met als output de 2x1 vector. 

In [17]:
def forward_pass(input_m, w_and_b):
    print(f"input_matrix is {input_m}")
    
    network_output = []
    
    for weights, bias in w_and_b:
        neuron_output = 0
        print(f"Weights of neuron: {weights}")
        print(f"Bias of neuron: {bias}")
        
        for w, v in zip(flatten_matrix(input_m), weights):
            print(f"Adding {w} times {v} to weighted_sum")
            neuron_output += (w*v)
            print(f"Weighted sum is now {neuron_output}")
        print(f"Adding bias of {bias} to weighted sum {neuron_output}")
        
        neuron_output += bias 
        network_output.append(neuron_output)
        print(f"Total neuron output is now {neuron_output}")
        
    return network_output
            

In [18]:
nn_outputs = forward_pass(cirkel_vector, w_and_b)

input_matrix is [[1], [1], [1], [1], [0], [1], [1], [1], [1]]
Weights of neuron: [0.38, 0.81, 0.96, 0.98, 0.56, 0.44, 0.82, 0.66, 0.94]
Bias of neuron: 0.68
Adding 1 times 0.38 to weighted_sum
Weighted sum is now 0.38
Adding 1 times 0.81 to weighted_sum
Weighted sum is now 1.19
Adding 1 times 0.96 to weighted_sum
Weighted sum is now 2.15
Adding 1 times 0.98 to weighted_sum
Weighted sum is now 3.13
Adding 0 times 0.56 to weighted_sum
Weighted sum is now 3.13
Adding 1 times 0.44 to weighted_sum
Weighted sum is now 3.57
Adding 1 times 0.82 to weighted_sum
Weighted sum is now 4.39
Adding 1 times 0.66 to weighted_sum
Weighted sum is now 5.05
Adding 1 times 0.94 to weighted_sum
Weighted sum is now 5.99
Adding bias of 0.68 to weighted sum 5.99
Total neuron output is now 6.67
Weights of neuron: [0.95, 0.17, 0.12, 0.33, 0.36, 0.89, 0.27, 0.32, 0.55]
Bias of neuron: 0.63
Adding 1 times 0.95 to weighted_sum
Weighted sum is now 0.95
Adding 1 times 0.17 to weighted_sum
Weighted sum is now 1.1199999

Met de bovenstaande code kunnen we dus een enkel-laags neuraal netwerk maken waarbij de output vector de waardes van de output laag geeft, en waarvan de lengte afhankelijk is van het aantal setjes weights en biases die we er in stoppen. Aangezien de enkele (output) laag uit twee neuronen bestaat verwachten we een vector van 2 x 1 als output:

In [19]:
print(nn_outputs)

[6.67, 4.2299999999999995]


In [20]:
# Checken of de numpy .dot methode dezelfde output geeft, en dat is zo. De handmatige manier werkt dus voor 
# dit netwerk met 1 laag.

x_o = np.dot(weight_list, flatten_matrix(cirkel_vector)) + bias_list

In [21]:
print(x_o)

[6.67 4.23]


Als laatste stap moeten we de output door een activatie functie halen, en in dit geval (waarbij er geen hidden layers in het netwerk bestaan, en we dus enkel de output layer hebben) is dat de softmax activatie functie. De softmax functie maakt van de "input" een verdeling met een totaal van 1, een waarschijnlijkheidsdistributie dus.

In [22]:
def softmax(vector):
    e = 2.71828
    output_vec = []
    vec = [e**a for a in vector]
    vec_sum = sum(vec)
    
    for n in vec:
        output_vec.append(n/vec_sum)
    
    return output_vec

In [23]:
print(softmax(nn_outputs))

[0.9198269667910991, 0.08017303320890098]


De bovenstaande output vector geeft ons dus de waarschijnlijkheid van de mogelijke uitkomsten, namelijk een kruis en een rondje. 

## Een class maken voor een layer

In [21]:
input_voorbeeld = flatten_matrix(cirkel)

print(input_voorbeeld)

[1, 1, 1, 1, 0, 1, 1, 1, 1]


In [1]:
layer_input = flatten_matrix(cirkel)

#Class aanmaken
class Layer:
    
    def __init__(self, neuron_amount, input_shape): # Aantal neuronen in de laag en het aa
        self.weights = np.random.randn(input_shape, neuron_amount) # We initialiseren de weight matrix
        self.biases = np.zeros((1, neuron_amount)) #biases zetten we op 0
        
    def forward_pass(self, inputs):
        
        self.output = np.dot(inputs, self.weights) + self.biases
        
    
    
    

NameError: name 'flatten_matrix' is not defined

SyntaxError: incomplete input (3066990509.py, line 1)